# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Saves random cities csv data to new data frame
gmap_df = pd.read_csv('random_cities.csv')
gmap_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,4845585,Westport,1,US,1612117089,45,41.1415,-73.3579,25.00,5.12
1,1,4946620,Palmer,75,US,1612117221,48,42.1584,-72.3287,23.00,4.61
2,2,2264557,Ponta do Sol,40,PT,1612117254,77,32.6667,-17.1000,64.40,3.44
3,3,1507390,Dikson,35,RU,1612117091,78,73.5069,80.5464,-27.85,8.70
4,4,2018735,Nyurba,58,RU,1612117255,76,63.2842,118.3319,-46.97,3.62
...,...,...,...,...,...,...,...,...,...,...,...
570,570,2412408,Nioro,66,GM,1612117320,8,13.3500,-15.7500,99.39,7.45
571,571,1608451,Nan,83,TH,1612117320,66,18.8333,100.6667,67.39,1.25
572,572,3385538,Vargem Grande,91,BR,1612117050,69,-3.5431,-43.9158,86.20,1.16
573,573,298846,Tunceli,75,TR,1612117172,75,39.1083,39.5471,39.20,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) #Load API key

#Establish Lat and Lng columns as coordinates
locations = gmap_df[['Lat','Lng']]
#Establish Humidity column with float values
humidity = gmap_df['Humidity'].astype(float)

#Set figure layout
figure_layout = {
    'width': '300px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
#Applies figure layout to figure
fig = gmaps.figure(layout=figure_layout)
#Creates heat layer and adds it to figure
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=True,
                                 max_intensity=7,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
gmap_df_2 = gmap_df.drop(gmap_df[(gmap_df['Max Temp']<70) | (gmap_df['Max Temp']>80)].index)
gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df_2['Wind Speed'] >= 10)].index)
gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df['Cloudiness'] != 0)].index)
#for i, city in gmap_df_2.iterrows():
#    try:
#        if (city[i,'Max Temp'] < 70 | city[i,'Max Temp'] > 80) & (city[i,'Wind Speed'] >= 10) & (city[i,'Cloudiness'] != 0):
#            city = []
#    except:
#        print('Data not available')
gmap_df_2

<ipython-input-4-53c02069cda3>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  gmap_df_2 = gmap_df_2.drop(gmap_df_2[(gmap_df['Cloudiness'] != 0)].index)


,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
25,25,1006984,East London,0,ZA,1612117230,88,-33.0153,27.9116,73.40,3.44
38,38,2070998,Geraldton,0,AU,1612117114,73,-28.7667,114.6000,71.60,8.05
143,143,986717,Kruisfontein,0,ZA,1612117124,74,-34.0033,24.7314,72.00,5.01
188,188,2444995,Filingué,0,NE,1612117274,15,14.3521,3.3168,78.48,8.66
222,222,1153850,Bo Phloi,0,TH,1612117278,51,14.3236,99.5161,78.01,3.44
374,374,3868707,Valdivia,0,CL,1612117088,44,-39.8142,-73.2459,77.00,4.61
445,445,2328952,Nguru,0,NG,1612117306,17,12.8791,10.4526,78.24,7.72
535,535,3932145,Pisco,0,PE,1612117308,61,-13.7000,-76.2167,77.00,8.05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
#Create hotel_df and create new column Hotel Name
hotel_df = gmap_df_2
hotel_df['Hotel Name'] = ''

#Base url and preliminary parameters for API call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {'type':'lodging','radius':5000,'key':g_key}

#Establishes for loop for lat/lng conversion and API calls
for i,row in hotel_df.iterrows():
    #Store Lat and Lng and insert into new dictionary key
    #called 'location'
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    #API request
    response = requests.get(base_url,params=params).json()
    try:
        #Tries to store first hotel name from response in
        #Hotel Name column
        hotel_df.loc[i, 'Hotel Name'] = response['results'][0]['name']
    except:
        #Puts 'No hotel found' if hotel not found
        hotel_df.loc[i, 'Hotel Name'] = 'No hotel found'
hotel_df

,Unnamed: 0,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
25,25,1006984,East London,0,ZA,1612117230,88,-33.0153,27.9116,73.40,3.44,Tu Casa
38,38,2070998,Geraldton,0,AU,1612117114,73,-28.7667,114.6000,71.60,8.05,Broadwater Mariner Resort
143,143,986717,Kruisfontein,0,ZA,1612117124,74,-34.0033,24.7314,72.00,5.01,Oyster Bay House Rental
188,188,2444995,Filingué,0,NE,1612117274,15,14.3521,3.3168,78.48,8.66,No hotel found
222,222,1153850,Bo Phloi,0,TH,1612117278,51,14.3236,99.5161,78.01,3.44,แอท เมืองพลอย รีสอร์ท At MuangPloy Resort
374,374,3868707,Valdivia,0,CL,1612117088,44,-39.8142,-73.2459,77.00,4.61,Hostal Entre Rios
445,445,2328952,Nguru,0,NG,1612117306,17,12.8791,10.4526,78.24,7.72,Government Lodge
535,535,3932145,Pisco,0,PE,1612117308,61,-13.7000,-76.2167,77.00,8.05,Hotel San Isidro Oficial


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…